# Train classification

Most sharing code train this dataset as a regression task.

But in this code, I train as a classification task.

I encode the target value pressure to 950 classes and calculate CrossEntropy Loss.

In [1]:
# from kaggle_secrets import UserSecretsClient
# secret_label = "wandb"
# secret_value = UserSecretsClient().get_secret(secret_label)
# # !wandb login $secret_value

In [1]:
import gc
import os
import random
# import wandb
import math

import numpy as np
import pandas as pd

from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import AdamW
from transformers import get_cosine_schedule_with_warmup
from sklearn.preprocessing import RobustScaler

device = torch.device("cuda")

In [2]:
class config:
    EXP_NAME = "exp068_categorical"
    
    INPUT = "/home/knikaido/work/Ventilator-Pressure-Prediction/data/ventilator-pressure-prediction"
    OUTPUT = "./output"
    N_FOLD = 5
    SEED = 0
    
    LR = 5e-3
    N_EPOCHS = 50
    EMBED_SIZE = 64
    HIDDEN_SIZE = 256
    BS = 512
    WEIGHT_DECAY = 1e-3

    USE_LAG = 4
    CATE_FEATURES = ['R_cate', 'C_cate', 'RC_dot', 'RC_sum']
    CONT_FEATURES = ['u_in', 'u_out', 'time_step'] + ['u_in_cumsum', 'u_in_cummean', 'area', 'cross', 'cross2']
    LAG_FEATURES = ['breath_time']
    LAG_FEATURES += [f'u_in_lag_{i}' for i in range(1, USE_LAG+1)]
    #LAG_FEATURES += [f'u_in_lag_{i}_back' for i in range(1, USE_LAG+1)]
    LAG_FEATURES += [f'u_in_time{i}' for i in range(1, USE_LAG+1)]
    #LAG_FEATURES += [f'u_in_time{i}_back' for i in range(1, USE_LAG+1)]
    LAG_FEATURES += [f'u_out_lag_{i}' for i in range(1, USE_LAG+1)]
    #LAG_FEATURES += [f'u_out_lag_{i}_back' for i in range(1, USE_LAG+1)]
    ALL_FEATURES = CATE_FEATURES + CONT_FEATURES + LAG_FEATURES
    
    NOT_WATCH_PARAM = ['INPUT']

In [3]:
def set_seed(seed=config.SEED):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [4]:
class VentilatorDataset(Dataset):
    
    def __init__(self, df, label_dic=None):
        self.dfs = [_df for _, _df in df.groupby("breath_id")]
        self.label_dic = label_dic
        
    def __len__(self):
        return len(self.dfs)
    
    def __getitem__(self, item):
        df = self.dfs[item]
        X = df[config.ALL_FEATURES].values
        y = df['pressure'].values
        if self.label_dic is None:
            label = [-1]
        else:
            label = [self.label_dic[i] for i in y]

        d = {
            "X": torch.tensor(X).float(),
            "y" : torch.tensor(y).float(),
        }
        return d

In [5]:
class VentilatorModel(nn.Module):
    
    def __init__(self):
        super(VentilatorModel, self).__init__()
        self.r_emb = nn.Embedding(3, 2, padding_idx=0)
        self.c_emb = nn.Embedding(3, 2, padding_idx=0)
        self.rc_dot_emb = nn.Embedding(8, 4, padding_idx=0)
        self.rc_sum_emb = nn.Embedding(8, 4, padding_idx=0)
        self.seq_emb = nn.Sequential(
            nn.Linear(12+len(config.CONT_FEATURES)+len(config.LAG_FEATURES), config.EMBED_SIZE),
            nn.LayerNorm(config.EMBED_SIZE),
        )
        
        self.lstm = nn.LSTM(config.EMBED_SIZE, config.HIDDEN_SIZE, batch_first=True, bidirectional=True, dropout=0.2, num_layers=4)

        self.head = nn.Sequential(
            nn.Linear(config.HIDDEN_SIZE * 2, config.HIDDEN_SIZE * 2),
            nn.LayerNorm(config.HIDDEN_SIZE * 2),
            nn.ReLU(),
            nn.Linear(config.HIDDEN_SIZE * 2, 1),
        )
        
        # Encoder
        initrange = 0.1
        self.r_emb.weight.data.uniform_(-initrange, initrange)
        self.c_emb.weight.data.uniform_(-initrange, initrange)
        self.rc_dot_emb.weight.data.uniform_(-initrange, initrange)
        self.rc_sum_emb.weight.data.uniform_(-initrange, initrange)
        
        # LSTM
        for n, m in self.named_modules():
            if isinstance(m, nn.LSTM):
                print(f'init {m}')
                for param in m.parameters():
                    if len(param.shape) >= 2:
                        nn.init.orthogonal_(param.data)
                    else:
                        nn.init.normal_(param.data)

    def forward(self, X, y=None):
        # embed
        bs = X.shape[0]
        r_emb = self.r_emb(X[:,:,0].long()).view(bs, 80, -1)
        c_emb = self.c_emb(X[:,:,1].long()).view(bs, 80, -1)
        rc_dot_emb = self.rc_dot_emb(X[:,:,2].long()).view(bs, 80, -1)
        rc_sum_emb = self.rc_sum_emb(X[:,:,3].long()).view(bs, 80, -1)
        
        seq_x = torch.cat((r_emb, c_emb, rc_dot_emb, rc_sum_emb, X[:, :, 4:]), 2)
        emb_x = self.seq_emb(seq_x)
        
        out, _ = self.lstm(emb_x, None) 
        logits = self.head(out)

        if y is None:
            loss = None
        else:
            loss = self.loss_fn(logits, y)
            
        return logits, loss
    
#     def loss_fn(self, y_pred, y_true):
#         loss = nn.CrossEntropyLoss()(y_pred.reshape(-1, 950), y_true.reshape(-1))
#         return loss
    
    def loss_fn(self, y_pred, y_true):
        loss = nn.L1Loss()(y_pred.reshape(-1), y_true.reshape(-1))
        return loss

In [6]:
def train_loop(model, optimizer, scheduler, loader):
    losses, lrs = [], []
    model.train()
    optimizer.zero_grad()
    for d in loader:
        out, loss = model(d['X'].to(device), d['y'].to(device))
        
        losses.append(loss.item())
        step_lr = np.array([param_group["lr"] for param_group in optimizer.param_groups]).mean()
        lrs.append(step_lr)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()

    return np.array(losses).mean(), np.array(lrs).mean()


def valid_loop(model, loader, target_dic_inv):
    losses, predicts = [], []
    model.eval()
    for d in loader:
        with torch.no_grad():
            out, loss = model(d['X'].to(device), d['y'].to(device))
#         out = torch.tensor([[target_dic_inv[j.item()] for j in i] for i in out.argmax(2)])
        losses.append(loss.item())
        predicts.append(out.cpu())

    return np.array(losses).mean(), torch.vstack(predicts).numpy().reshape(-1)

def test_loop(model, loader, target_dic_inv):
    predicts = []
    model.eval()
    for d in loader:
        with torch.no_grad():
            out, _ = model(d['X'].to(device))
#         out = torch.tensor([[target_dic_inv[j.item()] for j in i] for i in out.argmax(2)])
        predicts.append(out.cpu())

    return torch.vstack(predicts).numpy().reshape(-1)

In [7]:
def add_feature(df):
    df['time_delta'] = df.groupby('breath_id')['time_step'].diff().fillna(0)
    df['delta'] = df['time_delta'] * df['u_in']
    df['area'] = df.groupby('breath_id')['delta'].cumsum()

    df['cross']= df['u_in']*df['u_out']
    df['cross2']= df['time_step']*df['u_out']
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    df['one'] = 1
    df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
    df['u_in_cummean'] =df['u_in_cumsum'] / df['count']
    
    df = df.drop(['count','one'], axis=1)
    return df

def add_lag_feature(df):
    # https://www.kaggle.com/kensit/improvement-base-on-tensor-bidirect-lstm-0-173
    for lag in range(1, config.USE_LAG+1):
        df[f'breath_id_lag{lag}']=df['breath_id'].shift(lag).fillna(0)
        df[f'breath_id_lag{lag}same']=np.select([df[f'breath_id_lag{lag}']==df['breath_id']], [1], 0)

        # u_in 
        df[f'u_in_lag_{lag}'] = df['u_in'].shift(lag).fillna(0) * df[f'breath_id_lag{lag}same']
        #df[f'u_in_lag_{lag}_back'] = df['u_in'].shift(-lag).fillna(0) * df[f'breath_id_lag{lag}same']
        df[f'u_in_time{lag}'] = df['u_in'] - df[f'u_in_lag_{lag}']
        #df[f'u_in_time{lag}_back'] = df['u_in'] - df[f'u_in_lag_{lag}_back']
        df[f'u_out_lag_{lag}'] = df['u_out'].shift(lag).fillna(0) * df[f'breath_id_lag{lag}same']
        #df[f'u_out_lag_{lag}_back'] = df['u_out'].shift(-lag).fillna(0) * df[f'breath_id_lag{lag}same']

    # breath_time
    df['time_step_lag'] = df['time_step'].shift(1).fillna(0) * df[f'breath_id_lag{lag}same']
    df['breath_time'] = df['time_step'] - df['time_step_lag']

    drop_columns = ['time_step_lag']
    drop_columns += [f'breath_id_lag{i}' for i in range(1, config.USE_LAG+1)]
    drop_columns += [f'breath_id_lag{i}same' for i in range(1, config.USE_LAG+1)]
    df = df.drop(drop_columns, axis=1)

    # fill na by zero
    df = df.fillna(0)
    return df

c_dic = {10: 0, 20: 1, 50:2}
r_dic = {5: 0, 20: 1, 50:2}
rc_sum_dic = {v: i for i, v in enumerate([15, 25, 30, 40, 55, 60, 70, 100])}
rc_dot_dic = {v: i for i, v in enumerate([50, 100, 200, 250, 400, 500, 2500, 1000])}    

def add_category_features(df):
    df['C_cate'] = df['C'].map(c_dic)
    df['R_cate'] = df['R'].map(r_dic)
    df['RC_sum'] = (df['R'] + df['C']).map(rc_sum_dic)
    df['RC_dot'] = (df['R'] * df['C']).map(rc_dot_dic)
    return df

norm_features = config.CONT_FEATURES + config.LAG_FEATURES
def norm_scale(train_df, test_df):
    scaler = RobustScaler()
    all_u_in = np.vstack([train_df[norm_features].values, test_df[norm_features].values])
    scaler.fit(all_u_in)
    train_df[norm_features] = scaler.transform(train_df[norm_features].values)
    test_df[norm_features] = scaler.transform(test_df[norm_features].values)
    return train_df, test_df

In [8]:
def main():
    
    train_df = pd.read_csv(f"{config.INPUT}/train.csv")
    test_df = pd.read_csv(f"{config.INPUT}/test.csv")
    sub_df = pd.read_csv(f"{config.INPUT}/sample_submission.csv")
    oof = np.zeros(len(train_df))
    test_preds_lst = []

    target_dic = {v:i for i, v in enumerate(sorted(train_df['pressure'].unique().tolist()))}
    target_dic_inv = {v: k for k, v in target_dic.items()}

    gkf = GroupKFold(n_splits=config.N_FOLD).split(train_df, train_df.pressure, groups=train_df.breath_id)
    for fold, (_, valid_idx) in enumerate(gkf):
        train_df.loc[valid_idx, 'fold'] = fold

    train_df = add_feature(train_df)
    test_df = add_feature(test_df)
    train_df = add_lag_feature(train_df)
    test_df = add_lag_feature(test_df)
    train_df = add_category_features(train_df)
    test_df = add_category_features(test_df)
    train_df, test_df = norm_scale(train_df, test_df)

    test_df['pressure'] = -1
    test_dset = VentilatorDataset(test_df)
    test_loader = DataLoader(test_dset, batch_size=config.BS,
                             pin_memory=True, shuffle=False, drop_last=False, num_workers=os.cpu_count())
    
    for fold in range(config.N_FOLD):
        print(f'Fold-{fold}')
        train_dset = VentilatorDataset(train_df.query(f"fold!={fold}"), target_dic)
        valid_dset = VentilatorDataset(train_df.query(f"fold=={fold}"), target_dic)

        set_seed()
        train_loader = DataLoader(train_dset, batch_size=config.BS,
                                  pin_memory=True, shuffle=True, drop_last=True, num_workers=os.cpu_count(),
                                  worker_init_fn=lambda x: set_seed())
        valid_loader = DataLoader(valid_dset, batch_size=config.BS,
                                  pin_memory=True, shuffle=False, drop_last=False, num_workers=os.cpu_count())

        model = VentilatorModel()
        model.to(device)

        optimizer = AdamW(model.parameters(), lr=config.LR, weight_decay=config.WEIGHT_DECAY)
        num_train_steps = int(len(train_loader) * config.N_EPOCHS)
        num_warmup_steps = int(num_train_steps / 10)
        scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)

        uniqe_exp_name = f"{config.EXP_NAME}_f{fold}"
#         wandb.init(project='Ventilator', entity='trtd56', name=uniqe_exp_name, group=config.EXP_NAME)
#         wandb_config = wandb.config
#         wandb_config.fold = fold
#         for k, v in dict(vars(config)).items():
#             if k[:2] == "__" or k in config.NOT_WATCH_PARAM:
#                 continue
#             wandb_config[k] = v
#         wandb.watch(model)
        
        os.makedirs(f'{config.OUTPUT}/{config.EXP_NAME}', exist_ok=True)
        model_path = f"{config.OUTPUT}/{config.EXP_NAME}/ventilator_f{fold}_best_model.bin"
        
        valid_best_score = float('inf')
        valid_best_score_mask = float('inf')
        for epoch in tqdm(range(config.N_EPOCHS)):
            train_loss, lrs = train_loop(model, optimizer, scheduler, train_loader)
            valid_loss, valid_predict = valid_loop(model, valid_loader, target_dic_inv)
            valid_score = np.abs(valid_predict - train_df.query(f"fold=={fold}")['pressure'].values).mean()

            mask = (train_df.query(f"fold=={fold}")['u_out'] == 0).values
            _score = valid_predict - train_df.query(f"fold=={fold}")['pressure'].values
            valid_score_mask = np.abs(_score[mask]).mean()
            print(f"epoch = {epoch}, valid mask score = {valid_score_mask}:")

            if valid_score < valid_best_score:
                valid_best_score = valid_score
                torch.save(model.state_dict(), model_path)
                oof[train_df.query(f"fold=={fold}").index.values] = valid_predict

            if valid_score_mask < valid_best_score_mask:
                valid_best_score_mask = valid_score_mask

#             wandb.log({
#                 "train_loss": train_loss,
#                 "valid_loss": valid_loss,
#                 "valid_score": valid_score,
#                 "valid_best_score": valid_best_score,
#                 "valid_score_mask": valid_score_mask,
#                 "valid_best_score_mask": valid_best_score_mask,
#                 "learning_rate": lrs,
#             })
            
            torch.cuda.empty_cache()
            gc.collect()
        
        model.load_state_dict(torch.load(model_path))
        test_preds = test_loop(model, test_loader, target_dic_inv)
        test_preds_lst.append(test_preds)
        
        sub_df['pressure'] = test_preds
        sub_df.to_csv(f"{config.OUTPUT}/{config.EXP_NAME}/sub_f{fold}.csv", index=None)

        train_df['oof'] = oof
        train_df.to_csv(f"{config.OUTPUT}/{config.EXP_NAME}/oof.csv", index=None)
#         wandb.finish()

        del model, optimizer, scheduler, train_loader, valid_loader, train_dset, valid_dset
        torch.cuda.empty_cache()
        gc.collect()
    
    sub_df['pressure'] = np.stack(test_preds_lst).mean(0)
    sub_df.to_csv(f"{config.OUTPUT}/{config.EXP_NAME}/submission_mean.csv", index=None)

    sub_df['pressure'] = np.median(np.stack(test_preds_lst), axis=0)
    sub_df.to_csv(f"{config.OUTPUT}/{config.EXP_NAME}/submission_median.csv", index=None)
    
    # Post Processing: https://www.kaggle.com/snnclsr/a-dummy-approach-to-improve-your-score-postprocess
    unique_pressures = train_df["pressure"].unique()
    sorted_pressures = np.sort(unique_pressures)
    total_pressures_len = len(sorted_pressures)

    def find_nearest(prediction):
        insert_idx = np.searchsorted(sorted_pressures, prediction)
        if insert_idx == total_pressures_len:
            # If the predicted value is bigger than the highest pressure in the train dataset,
            # return the max value.
            return sorted_pressures[-1]
        elif insert_idx == 0:
            # Same control but for the lower bound.
            return sorted_pressures[0]
        lower_val = sorted_pressures[insert_idx - 1]
        upper_val = sorted_pressures[insert_idx]
        return lower_val if abs(lower_val - prediction) < abs(upper_val - prediction) else upper_val
    
    sub_df = pd.read_csv(f"{config.OUTPUT}/{config.EXP_NAME}/submission_mean.csv")
    sub_df["pressure"] = sub_df["pressure"].apply(find_nearest)
    sub_df.to_csv(f"{config.OUTPUT}/{config.EXP_NAME}/submission_mean_pp.csv", index=None)
    
    sub_df = pd.read_csv(f"{config.OUTPUT}/{config.EXP_NAME}/submission_median.csv")
    sub_df["pressure"] = sub_df["pressure"].apply(find_nearest)
    sub_df.to_csv(f"{config.OUTPUT}/{config.EXP_NAME}/submission_median_pp.csv", index=None)
    
    cv_score = train_df.apply(lambda x: abs(x['oof'] - x['pressure']), axis=1).mean()
    print("CV:", cv_score)

In [9]:
if __name__ == "__main__":
    main()

Fold-0
init LSTM(64, 256, num_layers=4, batch_first=True, dropout=0.2, bidirectional=True)


  0%|          | 0/50 [00:00<?, ?it/s]

epoch = 0, valid mask score = 0.9453852457629062:
epoch = 1, valid mask score = 0.5194634257366699:
epoch = 2, valid mask score = 0.39860576686053933:
epoch = 3, valid mask score = 0.3416326869521652:
epoch = 4, valid mask score = 0.31051688770593844:
epoch = 5, valid mask score = 0.2859712235129969:
epoch = 6, valid mask score = 0.265067004639667:
epoch = 7, valid mask score = 0.26538234830239704:
epoch = 8, valid mask score = 0.2774170608882531:
epoch = 9, valid mask score = 0.23979488175541994:
epoch = 10, valid mask score = 0.22913753610826065:
epoch = 11, valid mask score = 0.2335857793744477:
epoch = 12, valid mask score = 0.24218560024604402:
epoch = 13, valid mask score = 0.22183550076256728:
epoch = 14, valid mask score = 0.22219287013361985:
epoch = 15, valid mask score = 0.21858413479405833:
epoch = 16, valid mask score = 0.2276371927379565:
epoch = 17, valid mask score = 0.21236664509831268:
epoch = 18, valid mask score = 0.2007667542788717:
epoch = 19, valid mask score = 0

KeyboardInterrupt: 